<a href="https://colab.research.google.com/github/hyakuroume/Generative_AI/blob/develop/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## テキストのトークン化

In [1]:
from transformers import AutoTokenizer

In [46]:
# Qwenのトークナイザーに通してトークンを表す数値列に変換
prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids
input_ids

[2132, 572, 264, 6319, 323, 13458, 88]

In [47]:
for t in input_ids:
    print(t, "\t:", tokenizer.decode(t))

2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y


## 確率の予測

In [48]:
# 因果言語モデル(SmolLM)のロード
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

In [49]:
# pytorchのtensor形式で返すように指定してトークナイズ
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

In [50]:
outputs = model(input_ids)
outputs.logits.shape

torch.Size([1, 7, 151936])

In [51]:
# 最後のトークンに対応するトークンID
final_logits = model(input_ids).logits[0, -1]
final_logits.argmax()

tensor(3729)

In [52]:
# トークンIDを指定してでコード
tokenizer.decode(final_logits.argmax())

' night'

In [53]:
# その他のトークンの候補の表示
import torch

top10_logits = torch.topk(final_logits, 10)

for index in top10_logits.indices:
    print(tokenizer.decode(index))

 night
 evening
 day
 morning
 winter
 afternoon
 Saturday
 Sunday
 Friday
 October


In [54]:
# それぞれの候補トークンの確率
top10 = torch.topk(final_logits.softmax(dim=0), 10)
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10}{value.item():.2%}")

 night    88.71%
 evening  4.30%
 day      2.19%
 morning  0.49%
 winter   0.45%
 afternoon0.27%
 Saturday 0.25%
 Sunday   0.19%
 Friday   0.17%
 October  0.16%
